In [1]:
import pandas as pd 
import numpy as np
import datetime
import warnings
from tqdm import tqdm
from optimization import PSO
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('clear_data.csv', index_col=0, encoding='utf-32')

In [3]:
df.drop('flight_cat', axis=1, inplace=True)

#### Предсказываем как меняется мгновенная механическая скорость 

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
X = df.drop('Мгновенная механическая скорость бурения', axis=1)
y = df['Мгновенная механическая скорость бурения']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=123)

In [5]:
from catboost import CatBoostRegressor
model = CatBoostRegressor(iterations=5500, 
                          depth=20, 
                          learning_rate=0.1, 
                          loss_function='RMSE',
                          verbose=False)
model.fit(X_train, y_train)

In [6]:
y_pred = model.predict(X_test)
# mean_squared_error(y_pred, y_test) ** 0.5

In [10]:
feature_const = ['Давление промывочной жидкости', 'Крутящий момент на шпинделе','Глубина скважины (м)', 'plast']
feature_dynamic = ['Частота вращения шпинделя','Осевая нагрузка', 'Расход промывочной жидкости']

In [11]:
df[(df['Мгновенная механическая скорость бурения'] > 0) & (df['Глубина скважины (м)'] > 5)][feature_const].iloc[10]

Давление промывочной жидкости      1.011864
Крутящий момент на шпинделе      525.144068
Глубина скважины (м)              16.876271
plast                              0.000000
Name: 2022-03-22 03:25:00, dtype: float64

In [12]:
df[(df['Мгновенная механическая скорость бурения'] > 0) & (df['Глубина скважины (м)'] > 5)][feature_dynamic].iloc[10]

Частота вращения шпинделя      446.152542
Осевая нагрузка                  0.372322
Расход промывочной жидкости     22.033898
Name: 2022-03-22 03:25:00, dtype: float64

In [13]:
target = df[(df['Мгновенная механическая скорость бурения'] > 0) & (df['Глубина скважины (м)'] > 5)]['Мгновенная механическая скорость бурения'].iloc[10]

In [14]:
func = lambda x, y, z : - model.predict(np.array((1, 525, x, y, z, 16, 0)))

In [15]:
bound = ((0, 400), (0, 10), (10, 140))
var_name = ['Частота вращения шпинделя', 'Осевая нагрузка', 'Расход промывочной жидкости']
POP_SIZE = 100
w = 1
c1 = 0.2
c2 = 0.2
v_max = 0.05
 
pso = PSO(func, bound, POP_SIZE, w, c1, c2, v_max, var_name=var_name)

In [16]:
for _ in range(15):
    pso.pso()
    #print(pso.get_fitness().sum())
    
print(pso.global_best, func(*pso.global_best))

[137.82979884   8.35609691  74.79005096] 12.328625220795313


In [17]:
table = pso.info()
table[table['fitness'] == table['fitness'].max()]

,Частота вращения шпинделя,Осевая нагрузка,Расход промывочной жидкости,fitness
17,137.829799,8.356097,74.790051,12.328625
99,140.788106,8.504751,73.635814,12.328625


#### Сравнительная оптимизация

In [18]:
tg = df.columns[1]

In [133]:
const = df[(df['Мгновенная механическая скорость бурения'] > 0) & (df['Глубина скважины (м)'] > 5)][feature_const][10:40]
target = df[(df['Мгновенная механическая скорость бурения'] > 0) & (df['Глубина скважины (м)'] > 5)][tg][10:40]
dynamic = df[(df['Мгновенная механическая скорость бурения'] > 0) & (df['Глубина скважины (м)'] > 5)][feature_dynamic][10:40]

In [59]:
optimize_data = pd.DataFrame()
for j in tqdm(range(const.shape[0])):
    observe = const.iloc[j]
    func = lambda x, y, z : - model.predict(np.array((observe[0], observe[1], x, y, z, observe[2], observe[3])))
    #func = lambda x, y, z: -model.predict(np.array((x, y, z)))
    if j == 0:
        bound = ((0, 400), (0, 10), (10, 140))
    else:
        bound = ((optimize_data.iloc[-1][0] * 0.97, optimize_data.iloc[-1][0] * 1.03),
                (optimize_data.iloc[-1][1] * 0.9, optimize_data.iloc[-1][1] * 1.03),
                (optimize_data.iloc[-1][2] * 0.95, optimize_data.iloc[-1][2] * 1.1))
    var_name = ['Частота вращения шпинделя', 'Осевая нагрузка', 'Расход промывочной жидкости']
    POP_SIZE = 100
    w = 1
    c1 = 0.2
    c2 = 0.2
    v_max = 0.05
    pso = PSO(func, bound, POP_SIZE, w, c1, c2, v_max, var_name=var_name)
    for _ in range(25):
        pso.pso()
    #print(pso.global_best, func(*pso.global_best))
    table = pso.info()
#     if j > 0:
#         a = np.abs((table[table['fitness'] == table['fitness'].max()].iloc[0][-1] - optimize_data['fitness'].iloc[-1]))/optimize_data['fitness'].iloc[-1]
#         print(a)
    if j > 0 and a > 0.3:
        for _ in range(25 + 10):
            pso.pso()
    table = pso.info()   
    optimize_data = optimize_data.append(table[table['fitness'] == table['fitness'].max()].iloc[0])
optimize_data.reset_index(drop = True, inplace = True)

100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [06:20<00:00, 12.69s/it]


#### Результаты

In [62]:
optimize_data

,Частота вращения шпинделя,Осевая нагрузка,Расход промывочной жидкости,fitness
0,136.179992,6.201709,72.109195,12.328625
1,133.823674,5.581538,70.841885,13.238786
2,134.113444,5.050433,70.220529,13.290086
3,134.526552,4.596633,69.362998,12.253252
4,138.562349,4.136970,69.946540,6.310388
5,134.470213,3.767857,76.941194,6.371341
6,134.499533,3.409565,78.870634,6.371341
7,133.653272,3.178873,86.757697,7.573026
8,135.719200,3.190411,93.716748,8.371306
9,134.206752,3.189165,92.620621,8.371306


In [66]:
target

timestamps
2022-03-22 03:25:00     5.625424
2022-03-22 03:30:00     5.803333
2022-03-22 03:35:00     6.783333
2022-03-22 03:40:00     8.020000
2022-03-22 03:45:00     6.525581
2022-03-22 03:50:00     6.525581
2022-03-22 03:55:00     6.525581
2022-03-22 04:00:00     6.525581
2022-03-22 04:05:00     6.525581
2022-03-22 04:10:00     6.525581
2022-03-22 04:15:00     6.525581
2022-03-22 04:20:00     6.525581
2022-03-22 04:25:00     6.941176
2022-03-22 04:30:00     5.436000
2022-03-22 04:35:00     5.436000
2022-03-22 04:40:00     7.957143
2022-03-22 05:00:00    21.847458
2022-03-22 05:05:00     5.506250
2022-03-22 05:30:00     8.616667
2022-03-22 05:35:00     6.420339
2022-03-22 05:40:00     6.200000
2022-03-22 05:45:00     6.267797
2022-03-22 05:50:00     7.108333
2022-03-22 05:55:00     7.250000
2022-03-22 06:00:00     6.084615
2022-03-22 06:05:00     5.200000
2022-03-22 06:10:00     5.200000
2022-03-22 06:15:00     5.200000
2022-03-22 06:50:00     6.400000
2022-03-22 06:55:00     6.400000